In [1]:
import math
import string
import time
from collections import defaultdict
from helpful_functions import *

In [2]:
with open("day7_input.txt", "r") as f:
    prog_inp = f.readlines()

prog_inp = [line.rstrip() for line in prog_inp]
if prog_inp[-1] == '':
    del prog_inp[-1]
print(prog_inp[:10])

['pale chartreuse bags contain 3 faded orange bags.', 'drab gold bags contain 5 dark aqua bags.', 'mirrored magenta bags contain 3 dotted violet bags.', 'posh black bags contain 3 dark lavender bags, 3 mirrored coral bags, 1 dotted chartreuse bag.', 'striped yellow bags contain 5 pale red bags, 2 light lime bags, 5 clear indigo bags.', 'bright aqua bags contain 2 vibrant orange bags, 4 clear lavender bags, 1 pale gray bag.', 'dim silver bags contain 2 bright blue bags, 3 faded olive bags, 1 faded cyan bag.', 'light beige bags contain 2 bright silver bags, 4 vibrant crimson bags.', 'dull cyan bags contain 2 drab beige bags, 3 mirrored turquoise bags, 2 mirrored indigo bags.', 'dark black bags contain 5 dull turquoise bags, 4 faded cyan bags, 4 plaid coral bags.']


In [3]:
def str_def_dict(dd):
    c = "".join(group_by(str(dd), lambda v: v in "}{", join_as_str=True)[1:-1])
    return "{" + c + "}"

In [4]:
rules = dict()
bags = set()
contain_nothing = set()

for line in prog_inp:
    words = group_by(line, " ", join_as_str=True)
    split_line = group_by(words, lambda word: word == "contain" or ',' in word, inclusive="front")
    bag = tuple(split_line[0][:-2])
    containments = defaultdict(lambda v: 0)
    if split_line[1] == ['no', 'other', 'bags.']:
        contain_nothing.add(bag)
    else:
        for contained_bag in split_line[1:]:
            containments[tuple(contained_bag[1:-1])] = int(contained_bag[0])
        bags.add(bag)
    rules[bag] = containments

In [5]:
parsed = []
for (i, key) in enumerate(rules.keys()):
    parsed.append(str(key) + ' : ' + str_def_dict(rules[key]))
    if i < 10: print(parsed[-1])

with open("day7_output.txt", 'w') as f:
    f.write("\n".join(parsed))

('pale', 'chartreuse') : {('faded', 'orange'): 3}
('drab', 'gold') : {('dark', 'aqua'): 5}
('mirrored', 'magenta') : {('dotted', 'violet'): 3}
('posh', 'black') : {('dark', 'lavender'): 3, ('mirrored', 'coral'): 3, ('dotted', 'chartreuse'): 1}
('striped', 'yellow') : {('pale', 'red'): 5, ('light', 'lime'): 2, ('clear', 'indigo'): 5}
('bright', 'aqua') : {('vibrant', 'orange'): 2, ('clear', 'lavender'): 4, ('pale', 'gray'): 1}
('dim', 'silver') : {('bright', 'blue'): 2, ('faded', 'olive'): 3, ('faded', 'cyan'): 1}
('light', 'beige') : {('bright', 'silver'): 2, ('vibrant', 'crimson'): 4}
('dull', 'cyan') : {('drab', 'beige'): 2, ('mirrored', 'turquoise'): 3, ('mirrored', 'indigo'): 2}
('dark', 'black') : {('dull', 'turquoise'): 5, ('faded', 'cyan'): 4, ('plaid', 'coral'): 4}


In [6]:
shiny_gold_bag = ('shiny', 'gold')

In [7]:
def count_can_hold_shiny_gold():
    can_hold_shiny_gold = set()
    cannot_hold_shiny_gold = contain_nothing.copy()
    while 1:
        found_something = False
        for bag in bags:
            rule = rules[bag]
            needs = rule.keys()

            needs_met = True
            golds_needed = 0
            golds_possible = 0

            for need in needs:
                if need == shiny_gold_bag:
                    golds_needed += rule[need]

                elif need in cannot_hold_shiny_gold:
                    continue

                elif need in can_hold_shiny_gold:
                    golds_needed += rule[need]

                else:
                    needs_met = False
                    break

            if needs_met:
                if golds_needed == 0:
                    if not bag in cannot_hold_shiny_gold:
                        cannot_hold_shiny_gold.add(bag)
                        found_something = True
                else:
                    if not bag in can_hold_shiny_gold:
                        can_hold_shiny_gold.add(bag)
                        found_something = True
        if found_something == False: return len(can_hold_shiny_gold)

In [8]:
times = []
for i in range(500):
    t = time.time()
    nr_can_hold_shiny_gold = count_can_hold_shiny_gold()
    times.append(time.time() - t)
print(f"{nr_can_hold_shiny_gold} bags can hold a shiny gold bag")
print(f"it took {sum(times)/len(times)} seconds to run")

296 bags can hold a shiny gold bag
it took 0.005195801258087158 seconds to run


In [9]:
def can_hold(bag):
    rule = rules[bag]
    needs = rule.keys()
    if shiny_gold_bag in needs: return True
    for need in needs:
        if can_hold(need):
            return True
    return False

def count_can_hold_shiny_gold2():
    can_hold_shiny_gold = []
    for bag in bags:
        if can_hold(bag):
            can_hold_shiny_gold.append(bag)
    return len(can_hold_shiny_gold)

In [10]:
times = []
for i in range(100): # Only 100 runs because its ptryy slow
    t = time.time()
    nr_can_hold_shiny_gold = count_can_hold_shiny_gold2()
    times.append(time.time() - t)
print(f"{nr_can_hold_shiny_gold} bags can hold a shiny gold bag")
print(f"it took {sum(times)/len(times)} seconds to run")

296 bags can hold a shiny gold bag
it took 0.03737274169921875 seconds to run


In [11]:
def can_hold_opt(bag, ch, cnh):
    if bag in ch: return True
    if bag in cnh: return False
    rule = rules[bag]
    needs = rule.keys()
    if shiny_gold_bag in needs:
        ch.add(bag)
        return True
    for need in needs:
        if can_hold_opt(need, ch, cnh):
            ch.add(bag)
            return True
    cnh.add(bag)
    return False

def count_can_hold_shiny_gold2_opt():
    can_hold_shiny_gold = set()
    cannot_hold_shiny_gold = set()
    for bag in bags:
        can_hold_opt(bag, can_hold_shiny_gold, cannot_hold_shiny_gold)
    return len(can_hold_shiny_gold)

In [12]:
times = []
for i in range(500):
    t = time.time()
    nr_can_hold_shiny_gold = count_can_hold_shiny_gold2_opt()
    times.append(time.time() - t)
print(f"{nr_can_hold_shiny_gold} bags can hold a shiny gold bag")
print(f"it took {sum(times)/len(times)} seconds to run")

296 bags can hold a shiny gold bag
it took 0.0005048346519470214 seconds to run


In [13]:
# Part 2:

In [14]:
def bags_inside(bag):
    rule = rules[bag]
    needs = rule.keys()
    assert not shiny_gold_bag in needs, "infinite recursion"

    count = 0
    for need in needs:
        if need in contain_nothing:
            count += rule[need]
        else:
            count += (bags_inside(need) + 1) * rule[need]
    return count

In [15]:
times = []
for i in range(1000):
    t = time.time()
    nr_bags_inside = bags_inside(shiny_gold_bag)
    times.append(time.time() - t)
print(f"There must be {nr_bags_inside} bags inside a shiny gold bag")
print(f"it took {sum(times)/len(times)} seconds to run")

There must be 9339 bags inside a shiny gold bag
it took 1.678466796875e-05 seconds to run


In [16]:
def bags_inside2(bag, cache=dict()):
    if bag in cache: return cache[bag]
    rule = rules[bag]
    needs = rule.keys()
    assert not shiny_gold_bag in needs, "infinite recursion"

    count = 0
    for need in needs:
        if need in contain_nothing:
            count += rule[need]
        else:
            count += (bags_inside2(need, cache) + 1) * rule[need]
    cache[bag] = count
    return count

In [17]:
times = []
for i in range(1000):
    t = time.time()
    nr_bags_inside = bags_inside2(shiny_gold_bag)
    times.append(time.time() - t)
print(f"There must be {nr_bags_inside} bags inside a shiny gold bag")
print(f"it took {sum(times)/len(times)} seconds to run")

There must be 9339 bags inside a shiny gold bag
it took 3.1638145446777343e-07 seconds to run
